In [10]:
import tensorflow as tf
import numpy as np

In [2]:
CSV_COLUMNS = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
LABEL_COLUMN = 'Species'
COLUMNS = CSV_COLUMNS[:-1]

def parse_csv(line):
    fields = tf.decode_csv(line, [[1.0], [1.0], [1.0], [1.0], [1]])
    features = dict(zip(CSV_COLUMNS, fields))
    labels = features.pop(LABEL_COLUMN)
    return features, labels

def read_csv(filename, batch_size=2, repeat=None):
    return tf.data.TextLineDataset(filename).skip(1).map(parse_csv).shuffle(1000).repeat(repeat).batch(batch_size)

In [3]:
train_data_filename = "datasets/iris/train.csv"
test_data_filename = "datasets/iris/test.csv"

In [4]:
def input_fn_train():
    return read_csv(train_data_filename).make_one_shot_iterator().get_next()

def input_fn_test():
    return read_csv(test_data_filename, repeat=1).make_one_shot_iterator().get_next()

In [5]:
feature_columns = []
for column in COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(column))
print(feature_columns)

[_NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [6]:
classifier = tf.estimator.DNNClassifier(hidden_units=[10, 10], feature_columns=feature_columns, n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11d146d90>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpfMrWoo', '_global_id_in_cluster': 0, '_save_summary_steps': 100}


In [7]:
classifier.train(input_fn=input_fn_train, steps=2500)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpfMrWoo/model.ckpt.
INFO:tensorflow:loss = 7.375078, step = 1
INFO:tensorflow:global_step/sec: 753.557
INFO:tensorflow:loss = 1.1548603, step = 101 (0.134 sec)
INFO:tensorflow:global_step/sec: 1306.06
INFO:tensorflow:loss = 0.47095263, step = 201 (0.077 sec)
INFO:tensorflow:global_step/sec: 1243.7
INFO:tensorflow:loss = 0.928869, step = 301 (0.080 sec)
INFO:tensorflow:global_step/sec: 1276.5
INFO:tensorflow:loss = 0.38179848, step = 401 (0.078 sec)
INFO:tensorflow:global_step/sec: 1224.57
INFO:tensorflow:loss = 0.28186956, step = 501 (0.082 sec)
INFO:tensorflow:global_step/sec: 1329.29
INFO:tensorflow:loss = 0.2430818, step = 601 (0.075 sec)
INFO:tensorflow:

In [8]:
classifier.evaluate(input_fn=input_fn_test)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-24-19:50:44
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpfMrWoo/model.ckpt-2500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-24-19:50:44
INFO:tensorflow:Saving dict for global step 2500: accuracy = 0.96666664, average_loss = 0.07805012, global_step = 2500, loss = 0.15610024


{'accuracy': 0.96666664,
 'average_loss': 0.07805012,
 'global_step': 2500,
 'loss': 0.15610024}

In [9]:
# predict
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

def input_fn_predictions():
    return read_csv(test_data_filename, repeat=1).make_one_shot_iterator().get_next()
    return tf.data.Dataset.from_tensor_slices((predict_x, expected)).make_one_shot_iterator().get_next()

predictions = classifier.predict(input_fn=input_fn_predictions)

for pred_dict, expec in zip(predictions, expected):
    template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(SPECIES[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpfMrWoo/model.ckpt-2500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "Versicolor" (98.5%), expected "Setosa"

Prediction is "Versicolor" (99.6%), expected "Versicolor"

Prediction is "Setosa" (100.0%), expected "Virginica"
